# Understanding Tools in LLM Engineering

This notebook explains what tools are in LLM engineering, walks through code improvements, and completes the exercise of adding a tool to set ticket prices.

## Part 1: What is a Tool in LLM Engineering?

A **tool** in LLM engineering is a way to give a language model the ability to perform actions beyond just generating text. Think of it like giving the AI "hands" to interact with the real world.

### The Problem
LLMs can only generate text. They can't check databases, call APIs, or perform calculations in real-time.

### The Solution
We define "tools" (functions) that the LLM can request to be called. The workflow is:

1. User asks a question (e.g., "What's the price of a ticket to London?")
2. LLM recognizes it needs external data and says "I want to call the `get_ticket_price` function with `destination_city='London'`"
3. Your code actually executes that function
4. The result is sent back to the LLM
5. LLM formulates a natural language response using that data

### Key Components
- **Tool Definition**: A JSON schema describing the function name, what it does, and its parameters
- **Tool Handler**: Code that executes the actual function when the LLM requests it
- **Chat Loop**: Logic that detects when the LLM wants to use tools, handles it, and continues the conversation

## Part 2: Understanding the Code Improvements

The original code progressed through these stages:

| Version | Storage | Limitation |
|---------|---------|------------|
| Initial | Hardcoded dictionary | Prices can't be changed at runtime |
| Improved | SQLite database | Prices are persistent and can be updated |
| Final | SQLite + Auto-pricing | Unknown cities get prices auto-generated |

### Key Improvements:
1. **From dictionary to database**: Prices are stored in SQLite, making them persistent and modifiable
2. **Added `set_ticket_price` function**: Allows updating prices programmatically
3. **Database uses UPSERT**: The `ON CONFLICT...DO UPDATE` pattern means it inserts new cities or updates existing ones
4. **Auto-pricing for unknown cities**: When a city has no price, the system generates one and saves it for future queries

## Part 3: Complete Streamlined Code with Exercise Solution

Below is the complete, cleaned-up code with the exercise completed (adding a tool to set ticket prices) and auto-pricing for unknown cities.

### Step 1: Imports and Configuration

In [ ]:
import sqlite3
import json
import random
import openai
import gradio as gr

# Configuration
MODEL = "gpt-4o-mini"  # or your preferred model
DB = "prices.db"

system_message = """You are a helpful airline ticket assistant. You can:
1. Look up ticket prices to various destinations
2. Set or update ticket prices when requested

When a price is not available for a city, one will be automatically generated and saved.
In such cases, advise the user that this is a newly added route and they should check back
later for potential price updates or promotions.

Always be friendly and helpful to customers."""

### Step 2: Database Setup

We use SQLite to store ticket prices persistently. This allows prices to be updated and retrieved across sessions.

In [ ]:
def init_database():
    """Initialize the database with the prices table."""
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
        conn.commit()

# Initialize the database
init_database()

### Step 3: Tool Functions

These are the actual functions that perform real work. The LLM doesn't execute these directly - it requests them to be called, and our code executes them.

**Key Feature**: `get_ticket_price` now auto-generates and saves prices for unknown cities!

In [ ]:
def generate_random_price():
    """Generate a random ticket price between $299 and $2999."""
    return round(random.uniform(299, 2999), 2)


def get_ticket_price(city):
    """Retrieve the ticket price for a given city from the database.
    
    If the city doesn't exist, automatically generate a price,
    save it to the database, and return it with a flag indicating
    it's a newly added route.
    """
    print(f"TOOL CALLED: get_ticket_price for '{city}'", flush=True)
    
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        
        if result:
            # City exists - return the price
            return json.dumps({
                "status": "found",
                "city": city,
                "price": result[0],
                "message": f"The ticket price to {city} is ${result[0]:.2f}"
            })
        else:
            # City doesn't exist - generate price, save it, and return
            new_price = generate_random_price()
            cursor.execute(
                'INSERT INTO prices (city, price) VALUES (?, ?)',
                (city.lower(), new_price)
            )
            conn.commit()
            print(f"AUTO-GENERATED: New price for '{city}': ${new_price}", flush=True)
            
            return json.dumps({
                "status": "newly_added",
                "city": city,
                "price": new_price,
                "message": f"The ticket price to {city} is ${new_price:.2f}. This is a newly added route - please check back later for potential price updates or promotions."
            })


def set_ticket_price(city, price):
    """Set or update the ticket price for a given city in the database."""
    print(f"TOOL CALLED: set_ticket_price for '{city}' at ${price}", flush=True)
    
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        # UPSERT: Insert if new, update if exists
        cursor.execute(
            'INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?',
            (city.lower(), price, price)
        )
        conn.commit()
    
    return json.dumps({
        "status": "success",
        "city": city,
        "price": price,
        "message": f"Successfully set the ticket price to {city} to ${price:.2f}"
    })

### Step 4: Seed Initial Data

Let's add some initial ticket prices to our database.

In [ ]:
# Seed initial data
initial_prices = {"london": 799, "paris": 899, "tokyo": 1420, "sydney": 2999}

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    for city, price in initial_prices.items():
        cursor.execute(
            'INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?',
            (city, price, price)
        )
    conn.commit()

print("Database seeded with initial prices!")

### Step 5: Test the Functions Directly

In [ ]:
# Test get_ticket_price for existing city
print("Existing city (London):")
print(get_ticket_price("London"))
print()

# Test get_ticket_price for new city (will auto-generate price)
print("New city (Miami) - will auto-generate price:")
print(get_ticket_price("Miami"))
print()

# Query Miami again - now it exists!
print("Miami again - now it exists:")
print(get_ticket_price("Miami"))

In [ ]:
# Test set_ticket_price
print("Setting Berlin price to $550:")
print(set_ticket_price("Berlin", 550))
print()

print("Getting Berlin price:")
print(get_ticket_price("Berlin"))

### Step 6: Tool Definitions (JSON Schemas)

These JSON schemas tell the LLM what tools are available and how to use them.

In [ ]:
# Tool definition for getting ticket prices
get_price_function = {
    "name": "get_ticket_price",
    "description": """Get the price of a return ticket to the destination city.
    If the city doesn't have a price yet, one will be automatically generated and saved.
    The response includes a 'status' field: 'found' for existing prices, 'newly_added' for auto-generated prices.
    For newly added routes, advise the user to check back for potential price updates.""",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# Tool definition for setting ticket prices
set_price_function = {
    "name": "set_ticket_price",
    "description": "Set or update the price of a return ticket to a destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city to set the ticket price for",
            },
            "price": {
                "type": "number",
                "description": "The new price for the ticket in dollars",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    }
}

# Register both tools in a list
tools = [
    {"type": "function", "function": get_price_function},
    {"type": "function", "function": set_price_function}
]

### Step 7: Tool Handler

This function receives tool call requests from the LLM and executes the appropriate function.

In [ ]:
def handle_tool_calls(message):
    """Process all tool calls from the LLM's response."""
    responses = []
    
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        
        if tool_call.function.name == "get_ticket_price":
            city = arguments.get('destination_city')
            result = get_ticket_price(city)
        
        elif tool_call.function.name == "set_ticket_price":
            city = arguments.get('destination_city')
            price = arguments.get('price')
            result = set_ticket_price(city, price)
        
        else:
            result = json.dumps({"error": f"Unknown tool: {tool_call.function.name}"})
        
        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id
        })
    
    return responses

### Step 8: Chat Function (Main Conversation Loop)

In [ ]:
def chat(message, history):
    """Process a chat message, handling any tool calls the LLM makes."""
    # Build the complete message history
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = (
        [{"role": "system", "content": system_message}] 
        + history 
        + [{"role": "user", "content": message}]
    )
    
    # Get initial response from LLM
    response = openai.chat.completions.create(
        model=MODEL, 
        messages=messages, 
        tools=tools
    )
    
    # Process tool calls until LLM is done
    while response.choices[0].finish_reason == "tool_calls":
        assistant_message = response.choices[0].message
        tool_responses = handle_tool_calls(assistant_message)
        
        messages.append(assistant_message)
        messages.extend(tool_responses)
        
        response = openai.chat.completions.create(
            model=MODEL, 
            messages=messages, 
            tools=tools
        )
    
    return response.choices[0].message.content

### Step 9: Launch the Gradio Interface

In [ ]:
# Launch the chat interface
gr.ChatInterface(fn=chat, type="messages").launch()

## Part 4: Understanding the Workflow

### Workflow for Existing City:
```
User: "What's the price to London?"
  → LLM calls get_ticket_price("London")
  → Returns: {status: "found", price: 799}
  → LLM: "The ticket to London is $799.00"
```

### Workflow for New City (Auto-Pricing):
```
User: "What's the price to Dubai?"
  → LLM calls get_ticket_price("Dubai")
  → City not found → Generate random price ($1,547.32)
  → Save to database
  → Returns: {status: "newly_added", price: 1547.32}
  → LLM: "The ticket to Dubai is $1,547.32. This is a newly added 
          route - please check back later for potential price updates!"
```

### Next Query for Same City:
```
User: "What's the price to Dubai?"
  → LLM calls get_ticket_price("Dubai")
  → City found!
  → Returns: {status: "found", price: 1547.32}
  → LLM: "The ticket to Dubai is $1,547.32"
```

## Try These Example Prompts

Once the chat interface is running, try these:

**Existing cities:**
- "What's the ticket price to Paris?"
- "How much to fly to Tokyo?"

**New cities (will auto-generate prices):**
- "What's the price to Dubai?"
- "How much for a ticket to Mexico City?"
- "I want to go to Cape Town, what's the fare?"

**Then query them again:**
- "Actually, what was the Dubai price again?"

**Manual price setting:**
- "Set the price to Berlin to $550"
- "Update Tokyo's ticket to $1600"